In [6]:
%matplotlib qt5
import numpy as np
import matplotlib.pyplot as plt
import mahotas as mh
import os
import fnmatch

from scipy.ndimage import label
from scipy.ndimage import center_of_mass
from scipy.spatial import distance

CCD_AREA = 0.08602 #cm^2
IMG_PIXELS = 350208
SIGMA = 10


def create_plot(path):
    
    img = mh.imread(path)
    fig, axes = plt.subplots(ncols=4, figsize=(10, 5.5))
    threshold = mh.thresholding.otsu(img)

    img_filt = mh.gaussian_filter(img, SIGMA)
    labeled, nr_objects = label(img_filt > threshold)
    T_filt = mh.thresholding.otsu(img_filt.astype('uint8'))
    for k in range(1, nr_objects):
        location = str(center_of_mass(img, labeled, k))
        print("Object {} center of mass at {}".format(k, location))
        
    speckle_pixel = np.sum(np.sum(img > threshold))
    speckle_area = (speckle_pixel/IMG_PIXELS) * CCD_AREA

    # find distance between spekle
    distance_between_1_2 = distance.euclidean(center_of_mass(img, labeled, 1), center_of_mass(img, labeled, 2))
    distance_between_1_3 = distance.euclidean(center_of_mass(img, labeled, 1), center_of_mass(img, labeled, 3))
    print("Distace between 1 and 2 = {}".format(distance_between_1_2))
    print("Distace between 1 and 3 = {}".format(distance_between_1_3))
    axes[0].imshow(img, cmap=plt.cm.gray)
    axes[0].set_title("orginal image")
    axes[1].imshow(img > T_filt, cmap=plt.cm.gray)
    axes[1].set_title("Threshold = {0}".format(threshold))
    axes[2].imshow(img_filt, cmap=plt.cm.gray)
    axes[2].set_title("Gauss filt, $\sigma$ = {0}".format(SIGMA))
    
    axes[3].imshow(img_filt > T_filt, cmap=plt.cm.gray)
    
    x_12 = []
    y_12 = []
    x_12.append(center_of_mass(img, labeled, 1)[0])
    x_12.append(center_of_mass(img, labeled, 2)[0])
    y_12.append(center_of_mass(img, labeled, 1)[1])
    y_12.append(center_of_mass(img, labeled, 2)[1])
    
    x_13 = []
    y_13 = []
    x_13.append(center_of_mass(img, labeled, 1)[0])
    x_13.append(center_of_mass(img, labeled, 3)[0])
    y_13.append(center_of_mass(img, labeled, 1)[1])
    y_13.append(center_of_mass(img, labeled, 3)[1])
    
    axes[3].plot(y_12, x_12, 'r-', lw=3)
    axes[3].plot(y_13, x_13, 'r-', lw=3)
    axes[3].set_title("numbers of speckle = {0} \n $P = {1:.6f}  cm^2$".format(nr_objects, speckle_area))
    
cwd = os.getcwd()
create_plot(os.path.join(cwd, "test", '1T04633E.BMP'))

Object 1 center of mass at (63.131297069464665, 162.29226924977368)
Object 2 center of mass at (137.23286066989598, 645.26683306109419)
Object 3 center of mass at (324.67552633120886, 517.30600110593491)
Distace between 1 and 2 = 488.6261055416136
Distace between 1 and 3 = 440.95366385433084


NameError: name 'image_path' is not defined